<a href="https://colab.research.google.com/github/BraedenCu/NintendoSwitchScraper/blob/master/vexaiObjDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import pandas as pd
import pycocotools
import os
import torchvision
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T

In [ ]:
!git clone https://github.com/experiencor/raccoon_dataset.git
!pwd
labels = pd.read_csv("/content/raccoon_dataset/data/raccoon_labels.csv")

In [ ]:
!git clone https://github.com/pytorch/vision.git
!cd vision
!cp /content/vision/references/detection/utils.py ../
!cp /content/vision/references/detection/transforms.py ../
!cp /content/vision/references/detection/coco_eval.py ../
!cp /content/vision/references/detection/engine.py ../
!cp /content/vision/references/detection/coco_utils.py ../
%cd ../..

Cloning into 'vision'...
remote: Enumerating objects: 22445, done.
remote: Counting objects: 100% (3226/3226), done.
remote: Compressing objects: 100% (803/803), done.
remote: Total 22445 (delta 2471), reused 3038 (delta 2343), pack-reused 19219
Receiving objects: 100% (22445/22445), 27.08 MiB | 21.00 MiB/s, done.
Resolving deltas: 100% (16530/16530), done.
/


In [ ]:
def parse_one_annot(path_to_data_file, filename):
   data = pd.read_csv(path_to_data_file)
   boxes_array = data[data["filename"] == filename][["xmin", "ymin",        
   "xmax", "ymax"]].values
   return boxes_array

class RaccoonDataset(torch.utils.data.Dataset):
  def __init__(self, root, data_file, transforms=None):
    self.root = root
    self.transforms = transforms
    self.imgs = sorted(os.listdir(os.path.join(root, "images")))
    self.path_to_data_file = data_file
  def __getitem__(self, idx):
    # load images and bounding boxes
    img_path = os.path.join(self.root, "images", self.imgs[idx])
    img = Image.open(img_path).convert("RGB")
    box_list = parse_one_annot(self.path_to_data_file, self.imgs[idx])
    boxes = torch.as_tensor(box_list, dtype=torch.float32)
    num_objs = len(box_list)
    # there is only one class
    labels = torch.ones((num_objs,), dtype=torch.int64)
    image_id = torch.tensor([idx])
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,
    0])
    # suppose all instances are not crowd
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["image_id"] = image_id
    target["area"] = area
    target["iscrowd"] = iscrowd
    if self.transforms is not None:
      img, target = self.transforms(img, target)
    return img, target

  def __len__(self):
    return len(self.imgs)

In [ ]:
dataset = RaccoonDataset(root= "/content/raccoon_dataset",
data_file = "/content/raccoon_dataset/data/raccoon_labels.csv")
dataset.__getitem__(0)

In [ ]:
def get_model(num_classes):
   # load an object detection model pre-trained on COCO
   model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
   # get the number of input features for the classifier
   in_features = model.roi_heads.box_predictor.cls_score.in_features
   # replace the pre-trained head with a new on
   model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
   
   return model

def get_transform(train):
   transforms = []
   # converts the image, a PIL image, into a PyTorch Tensor
   transforms.append(T.ToTensor())
   if train:
      # during training, randomly flip the training images
      # and ground-truth for data augmentation
      transforms.append(T.RandomHorizontalFlip(0.5))
   return T.Compose(transforms)

In [ ]:
# use our dataset and defined transformations
dataset = RaccoonDataset(root= "/content/raccoon_dataset", data_file= "content/raccoon_dataset/data/raccoon_labels.csv",transforms = get_transform(train=True))
dataset_test = RaccoonDataset(root= "content/raccoon_dataset", data_file= "content/raccoon_dataset/data/raccoon_labels.csv", transforms = get_transform(train=False))
# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-40])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-40:])
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
              dataset, batch_size=2, shuffle=True, num_workers=4,
              collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(
         dataset_test, batch_size=1, shuffle=False, num_workers=4,
         collate_fn=utils.collate_fn)
print("We have: {} examples, {} are training and {} testing".format(len(indices), len(dataset), len(dataset_test)))

We have: 200 examples, 160 are training and 40 testing


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
torch.cuda.is_available()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# our dataset has two classes only - raccoon and not racoon
num_classes = 2
# get the model using our helper function
model = get_model(num_classes)
# move model to the right device
model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler which decreases the learning rate by # 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10
for epoch in range(num_epochs):
   # train for one epoch, printing every 10 iterations
   train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
   lr_scheduler.step()
   # evaluate on the test dataset
   evaluate(model, data_loader_test, device=device)

os.mkdir("content/pytorch-object-detection/raccoon/")
torch.save(model.state_dict(), "content/pytorch object detection/raccoon/model")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [0]  [ 0/80]  eta: 0:54:54  lr: 0.000068  loss: 0.8298 (0.8298)  loss_classifier: 0.6909 (0.6909)  loss_box_reg: 0.1348 (0.1348)  loss_objectness: 0.0022 (0.0022)  loss_rpn_box_reg: 0.0018 (0.0018)  time: 41.1822  data: 0.5112
